In [ ]:
using Revise
using PolynomialRings
using MatrixFactorizations
using OrbifoldEquivalence
@ring! ℤ[x,y,u,v,x₁,y₁]

In [ ]:
function A_D(d, A_vars, D_vars)
    _, (u,v,x,y) = polynomial_ring(A_vars..., D_vars..., basering=BigInt)
    
    f = div(x^d - u^2d, [x - u^2])[1]
    L = [0 x-u^2; f - y^2 0]
    R = [0 v-u*y; v+u*y 0]
    return L⨶R
end
Q = A_D(2, [:u,:v], [:x,:y])

In [ ]:
Q^2

In [ ]:
M = Q(x=x₁,y=y₁) ⨶ dual(Q)
M^2

In [ ]:
λ_u = -eye(Q(x=x₁,y=y₁)) ⨷ diff(dual(Q), :u) //4; power_of_u = 3
λ_v = -eye(Q(x=x₁,y=y₁)) ⨷ diff(dual(Q), :v) //2; power_of_v = 1

In [ ]:
inflated(M) = matrix_over_subring(matrix_over_subring(M, :u, power_of_u, :t₁), :v, power_of_v, :t₂)

inflated_M = inflated(M)

∇(X, t) = diff.(X, t)
At = (∇(inflated_M, :t₁) * ∇(inflated_M, :t₂) - ∇(inflated_M, :t₂) * ∇(inflated_M, :t₁))(t₁=0, t₂=0)//2

e = (inflated(λ_u) * inflated(λ_v))(t₁=0, t₂=0) * At

MM = inflated_M(t₁=0,t₂=0)
e^2 == e, MM*e == e*MM

In [ ]:
image_basis = hcat(gröbner_basis(columns(e))...)

In [ ]:
MM_reduced = matrix_solve_affine(MM_reduced -> image_basis*MM_reduced, MM*image_basis, (8,8))

In [ ]:
topright(MM_reduced)

In [ ]:
MM_simple = MM_reduced |> RowOp(2, 1//2*y, 1) |> ColOp(5, 1//2*y, 6)

In [ ]:
MM_simple^2